### Import dependencies

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
#!pip install squarify
#plt.style.use('fivethirtyeight')
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.tools as tls
import pygsheets
%matplotlib inline


### Authorize connection between Jupyter notebook (python working environment; like R studio) and google sheets so we can run analyses as the sheet is updated

In [2]:
gc = pygsheets.authorize() 
# Use customized credentials 
gc = pygsheets.authorize(client_secret='client_secret.json')
# For the first time, it will may produce as a link to authorize

### Open spreadsheet by name

In [3]:
sh = gc.open('PythonCurrentFlow_Aging')

### Open "Data" worksheet (can also use sh.sheet1)

In [4]:
data = sh[0]

### Get worksheet values as pandas dataframe

In [5]:
aging_data = pd.DataFrame(data.get_all_records())

In [6]:
aging_data

,Genus,Species,Sample ID,Species Code,Site,River,Basin,Lat,Long,Date Collected,...,Status Upon Collection,Final Age,Age dif,Z age,L age,B age,K&M Age,K age,M age,Notes
0,Amblema,plicata,1,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,15,14,
1,Amblema,plicata,2,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,16,,,,,16,16,12,
2,Amblema,plicata,3,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,17,,,,,17,17,9,
3,Amblema,plicata,4,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,13,12,
4,Amblema,plicata,5,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,14,,,,,14,14,12,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Lampsilis,teres,471,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
470,Lampsilis,teres,472,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
471,Lampsilis,teres,473,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
472,Lampsilis,teres,474,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,B: how could you tell difference b/w 0.5 & 1?


#### Rename columns 

In [7]:
df_rename=aging_data.rename(columns={"Sample ID": "id", "Genus":"genus", "Species":"species", "Species Code": "code", "Site":"site", "River":"river", "Basin":"basin", "Lat":"lat", "Long":"long", "Date Collected":"date", "Status Upon Collection":"status", "Ager":"ager", "Age":"age"})
df_rename

,genus,species,id,code,site,river,basin,lat,long,date,...,status,Final Age,Age dif,Z age,L age,B age,K&M Age,K age,M age,Notes
0,Amblema,plicata,1,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,15,14,
1,Amblema,plicata,2,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,16,,,,,16,16,12,
2,Amblema,plicata,3,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,17,,,,,17,17,9,
3,Amblema,plicata,4,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,13,12,
4,Amblema,plicata,5,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,14,,,,,14,14,12,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Lampsilis,teres,471,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
470,Lampsilis,teres,472,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
471,Lampsilis,teres,473,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,
472,Lampsilis,teres,474,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,...,Alive,0.5,0,0.5,0.5,1,,,,B: how could you tell difference b/w 0.5 & 1?


### What do age measurements for L. teres look like across Z, L & B?
#### Compare value counts 

In [8]:
pd.crosstab(index=aging_data['Species']=='teres', columns=aging_data['Z age'])

Z age,0.5,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,12,13,
Species,,,,,,,,,,,,,
False,0,1,1,0,0,0,0,0,1,0,1,1,320
True,7,6,12,33,22,18,14,10,7,2,0,0,16


##### L. teres
#### Ager Z: aged most mussels at 3 yrs old

In [9]:
pd.crosstab(index=aging_data['Species']=='teres', columns=aging_data['L age'])

L age,0.5,1,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,13,
Species,,,,,,,,,,,,,,
False,0,1,1,0,0,0,0,0,0,1,1,0,1,320
True,7,0,6,19,26,34,17,8,10,1,2,1,0,18


##### L. teres
#### Ager L: aged most mussels at 5 yrs old

In [10]:
pd.crosstab(index=aging_data['Species']=='teres', columns=aging_data['B age'])

B age,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,,,
Species,,,,,,,,,,,,
False,0,0,0,0,0,0,0,0,0,325,0,0
True,9,12,25,27,21,12,10,5,2,24,1,1


##### L. teres
#### Ager B: aged most mussels at 4 yrs old

In [11]:
pd.crosstab(index=aging_data['Species']=='teres', columns=aging_data['Site'])

Site,"Altair, TX","Bay City, TX","Bellville, TX","Gonzales, TX","Navasota, TX","Simonton, TX","Victoria, TX"
Species,,,,,,,
False,100,0,5,101,19,0,100
True,13,67,19,0,0,50,0


### Take a look at data (.head shows first 5 rows)

In [12]:
df_rename.head()

,genus,species,id,code,site,river,basin,lat,long,date,...,status,Final Age,Age dif,Z age,L age,B age,K&M Age,K age,M age,Notes
0,Amblema,plicata,1,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,15,14,
1,Amblema,plicata,2,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,16,,,,,16,16,12,
2,Amblema,plicata,3,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,17,,,,,17,17,9,
3,Amblema,plicata,4,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,15,,,,,15,13,12,
4,Amblema,plicata,5,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,...,Alive,14,,,,,14,14,12,


## Clean data
### Drop some columns

In [13]:
drop_df=df_rename.drop(columns=['Final Age', 'Age dif', 'K&M Age', 'K age', 'M age', 'Notes'])
drop_df.head()

,genus,species,id,code,site,river,basin,lat,long,date,Length (mm),status,Z age,L age,B age
0,Amblema,plicata,1,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,74.0,Alive,,,
1,Amblema,plicata,2,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,72.0,Alive,,,
2,Amblema,plicata,3,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,76.0,Alive,,,
3,Amblema,plicata,4,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,76.0,Alive,,,
4,Amblema,plicata,5,D,"Gonzales, TX",Guadalupe,Guadalupe,29.493646°,-97.431293°,9/24/19,77.0,Alive,,,


### Select teres data (b/c X, L and B have aged)

In [14]:
teres_df = drop_df.loc[drop_df["species"] == "teres"]
teres_df

,genus,species,id,code,site,river,basin,lat,long,date,Length (mm),status,Z age,L age,B age
295,Lampsilis,teres,297,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,128.0,Alive,6,6,
296,Lampsilis,teres,298,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,95.0,Alive,,,
297,Lampsilis,teres,299,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,81.0,Alive,,,
298,Lampsilis,teres,300,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,125.0,Alive,5,5,
299,Lampsilis,teres,301,E,"Altair, TX",Colorado,Colorado,29.595299°,-96.453229°,10/3/19,115.0,Alive,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Lampsilis,teres,471,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,32.5,Alive,0.5,0.5,1
470,Lampsilis,teres,472,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,35.5,Alive,0.5,0.5,1
471,Lampsilis,teres,473,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,33.0,Alive,0.5,0.5,1
472,Lampsilis,teres,474,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,19.0,Alive,0.5,0.5,1


### Drop NaNs

In [15]:
nan_value=float("NaN")
teres_df.replace("", nan_value, inplace=True)
clean_teres=teres_df.dropna()
clean_teres

/Users/biancahernandez/opt/anaconda2/envs/mussel_aging/lib/python3.8/site-packages/pandas/core/frame.py:4379: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,genus,species,id,code,site,river,basin,lat,long,date,Length (mm),status,Z age,L age,B age
334,Lampsilis,teres,336,E,"Bay City, TX",Colorado,Colorado,29.126034°,-96.044864°,5/5/20,139.0,Alive,5.0,5.0,5
335,Lampsilis,teres,337,E,"Bay City, TX",Colorado,Colorado,29.126034°,-96.044864°,5/5/20,121.0,Alive,6.0,11.0,6
337,Lampsilis,teres,339,E,"Bay City, TX",Colorado,Colorado,29.126034°,-96.044864°,5/5/20,107.0,Alive,4.0,5.0,4
339,Lampsilis,teres,341,E,"Bay City, TX",Colorado,Colorado,29.126034°,-96.044864°,5/5/20,114.0,Alive,4.0,4.0,5
340,Lampsilis,teres,342,E,"Bay City, TX",Colorado,Colorado,29.126034°,-96.044864°,5/5/20,74.0,Alive,1.0,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Lampsilis,teres,471,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,32.5,Alive,0.5,0.5,1
470,Lampsilis,teres,472,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,35.5,Alive,0.5,0.5,1
471,Lampsilis,teres,473,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,33.0,Alive,0.5,0.5,1
472,Lampsilis,teres,474,E,"Bellville, TX",Brazos,Brazos,29.939875°,-96.129332°,8/28/20,19.0,Alive,0.5,0.5,1


In [16]:
clean_teres['site'].value_counts()

Bay City, TX     57
Simonton, TX     49
Bellville, TX    19
Name: site, dtype: int64

In [17]:
clean_teres['species'].value_counts()

teres    125
Name: species, dtype: int64

### Download dependencies for figures

In [18]:
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.figure_factory as ff

In [55]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=clean_teres["Z age"], name='Z'))
fig.add_trace(go.Histogram(x=clean_teres["L age"], name='L'))
fig.add_trace(go.Histogram(x=clean_teres["B age"], name='B'))


# Overlay both histograms
fig.update_layout(
    
    title=dict(
        text='L. teres age counts (Z, L & B)',
        x=0.5,
        y=0.95,
        xanchor='center',
        yanchor= 'top',
        font=dict(
            size=16,
            color='#000000'
                )),
    barmode='group',
    bargap=0,
    template='simple_white',
#     paper_bgcolor='#FFFFFF',
#     plot_bgcolor='#FFFFFF',
    width=900, 
    height=500,
    bargroupgap=0
    )
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.update_xaxes(
        showgrid=False, 
        zeroline=False,
        title_text = "age"
        )
fig.update_yaxes(
        showgrid=False, 
        zeroline=False,
        title_text = "count",
        )

fig.show()

In [20]:
!pip install jupyterlab_dash
!jupyter labextension install jupyterlab-dash@0.1.0-alpha.3
!jupyter labextension install jupyterlab-dash

Building jupyterlab assets
Building jupyterlab assets


In [40]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

In [54]:
def teres_fig():
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=clean_teres["Z age"], name='Z'))
    fig.add_trace(go.Histogram(x=clean_teres["L age"], name='L'))
    fig.add_trace(go.Histogram(x=clean_teres["B age"], name='B'))


    # Overlay both histograms
    fig.update_layout(
    
        title=dict(
            text='L. teres age counts (Z, L & B)',
            x=0.5,
            y=0.95,
            xanchor='center',
            yanchor= 'top',
            font=dict(
                size=16,
                color='#000000'
                )),
        barmode='group',
        bargap=0,
        template='simple_white',
        # paper_bgcolor='#FFFFFF',
        # plot_bgcolor='#FFFFFF',
        width=900, 
        height=500,
        bargroupgap=0
        )
    # Reduce opacity to see both histograms
    fig.update_traces(opacity=0.75)
    fig.update_xaxes(
        showgrid=False, 
        zeroline=False,
        title_text = "age"
        )
    fig.update_yaxes(
        showgrid=False, 
        zeroline=False,
        title_text = "count",
        )

    return fig

#Build App
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])
app.layout = dbc.Jumbotron([
    dbc.Container([
    html.Br(),
    html.H1("Mussel aging project"),
    html.Br(),
    dbc.Row([dcc.Graph(id='teres',
              figure=teres_fig())
             
        
            ], style={"justify":"center"}),
    ],fluid=True, style={'textAlign': 'left'}),
    
    
])
# Run app and display result inline in the notebook
app.run_server(mode='external')   

Dash app running on http://127.0.0.1:8050/
